# Noiseless QCNN demo for Model3

This is noiseless QCNN demonstration for Model 3. 

The objective of this demonstration is different from Model 1/2 QCNN demos.

Model 1/2 QCNN demo compares QCNN performance with / without pre-training the quantum embedding.

Instead for Model 3 (as Model 3 can't be used without pre-traing unlike Model 1), I will compare QCNN performances between

1) Train CNN First with Model3_Fidelity. Then train parameterized QCNN.

2) Train parameterized embedding + parameterized QCNN all together as one optimization problem.


# 1. Load Noiseless Device

In [3]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

dev = qml.device('default.qubit', wires=4)

# 2. Feature Mapping Circuit

In [4]:

N_layers = 3

def Four_QuantumEmbedding2(input):
    for i in range(N_layers):
        for j in range(4):
            qml.Hadamard(wires=j)
            embedding.exp_Z(input[j], wires=j)
        for k in range(3):
            embedding.exp_ZZ1(input[4+k], wires=[k, k+1])
        embedding.exp_ZZ1(input[7], wires=[3,0])                       

def Four_QuantumEmbedding2_inverse(input):
    for i in range(N_layers):
        embedding.exp_ZZ1(input[7], wires=[3,0], inverse=True) 
        for k in reversed(range(3)):
            embedding.exp_ZZ1(input[k+4], wires=[k,k+1], inverse=True)
        qml.Barrier()
        for j in range(4):
            embedding.exp_Z(input[j], wires=j, inverse=True)
            qml.Hadamard(wires=j)

In [ ]:
@qml.qnode(dev, interface="torch")
def circuit3(inputs): 
    Four_QuantumEmbedding2(inputs[0:8])
    Four_QuantumEmbedding2_inverse(inputs[8:16])
    return qml.probs(wires=range(4))

class Model3_Fidelity(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Layer1: 28 * 28 -> 14 * 14
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 8
        self.fc = torch.nn.Linear(7 * 7, 8, bias=True)

        self.qlayer3 = qml.qnn.TorchLayer(circuit3, weight_shapes={})

    def forward(self, x1, x2):
        x1 = self.layer1(x1)
        x1 = self.layer2(x1)
        x1 = x1.view(-1, 7 * 7)
        x1 = self.fc(x1)

        x2 = self.layer1(x2)
        x2 = self.layer2(x2)
        x2 = x2.view(-1, 7 * 7)
        x2 = self.fc(x2)

        x = torch.concat([x1, x2], 1)
        x = self.qlayer3(x)
        return x[:,0]

class Model3_HSinner(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.matrix_fn2 = qml.matrix(circuit2)
        # Layer1: 28 * 28 -> 14 * 14
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 16
        self.fc = torch.nn.Linear(7 * 7, 16, bias=True)


    def forward(self, x1, x2):
        x1 = self.layer1(x1)
        x1 = self.layer2(x1)
        x1 = x1.view(-1, 7 * 7)
        x1 = self.fc(x1)

        x2 = self.layer1(x2)
        x2 = self.layer2(x2)
        x2 = x2.view(-1, 7 * 7)
        x2 = self.fc(x2)

        
        x = torch.concat([x1, x2], 1).to("cpu")
        x = [torch.real(torch.trace(self.matrix_fn2(a))) for a in x]
        x = torch.stack(x, dim=0).to(device)
        return x / 2**8